# Constraining the A.v. model main carbon metabolism 

There are particular constraints of A.v. that are currently not present in the model such as the useage of the Entner–Doudoroff Pathway (ED) and the pentose phosphate pathway (PPP) as well as sucrose metabolism particularly fructose catabolism. 

While we have MFA results (Wu et al. 2019) to constrain the model but we would like to use that data in the future to test the model so we will try to constrain flux to the ED and PP pathways with gene knockout first.  

The main supporting texts ###Finish###







In [1]:
import cobra.test
import pandas
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.io import write_sbml_model
from cobra.io import read_sbml_model

import numpy as np
import pandas as pd




In [2]:
#add model
model = read_sbml_model("../Data/Models/iAA1300.xml")
model.medium


{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 0.05,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 0.05,
 'EX_glc__D_e': 5.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 5.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_n2_e': 1000.0}

In [3]:
#Lets set gluose uptake to experimentally determined number 15 mmol (Wu et al) and make it diazotrophic by setting ammonia uptake to zero
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15

model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_n2_glu_15.csv')


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005112,0,0.00%
cl_e,EX_cl_e,0.005112,0,0.00%
cobalt2_e,EX_cobalt2_e,2.455E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006963,0,0.00%
fe2_e,EX_fe2_e,0.008105,0,0.00%
fe3_e,EX_fe3_e,0.007668,0,0.00%
glc__D_e,EX_glc__D_e,15,6,100.00%
h_e,EX_h_e,1.225,0,0.00%
k_e,EX_k_e,0.1917,0,0.00%
mg2_e,EX_mg2_e,0.00852,0,0.00%


Lets check out the model in Escher

In [4]:
import escher
from escher import Builder
from time import sleep

In [5]:


 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/central_metabolism.json',
                  reaction_data = solution.fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 0.0002190118504573584, 'CPPPGO': 0.0002190118504573584, 'GTH…

NameError: name 'builder' is not defined

In [7]:
#Lets set gluose uptake to experimentally determined number ~15 mmol (Wu et al) and make it diazotrophic by setting ammonia uptake to zero
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15

model.reactions.get_by_id("EX_nh4_e").lower_bound = -1000
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_nh4_glu_15.csv')


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.007469,0,0.00%
cl_e,EX_cl_e,0.007469,0,0.00%
cobalt2_e,EX_cobalt2_e,3.587E-05,0,0.00%
cu2_e,EX_cu2_e,0.001017,0,0.00%
fe2_e,EX_fe2_e,0.02305,0,0.00%
glc__D_e,EX_glc__D_e,15,6,100.00%
k_e,EX_k_e,0.2801,0,0.00%
mg2_e,EX_mg2_e,0.01245,0,0.00%
mn2_e,EX_mn2_e,0.0009915,0,0.00%
mobd_e,EX_mobd_e,1.004E-05,0,0.00%


As we can see the growth rate is far to fast for both diazotrophic and ammonia supplemented with a experimental glucose uptake rate. 

First we will look at the annotate PFK gene in A.v.:

While this gene Avin_23410 does have some homology to the E coli PFK2 enzyme it is also has high homology to 1-P fructose kinases and other genes in a simple blast search. We cannont be sure that this PFK is active in the foward direction making 1-6-fru in the EMP pathway. 

In [ ]:
model.reactions.EX_n2_e

In [ ]:
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15


model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

model.reactions.get_by_id("PFK").lower_bound = 0
model.reactions.get_by_id("PFK").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_n2_glu_15.csv')


model.summary()